In [19]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
iris = datasets.load_iris()

X = iris.data
y = iris.target



In [20]:
class Node():
    def __init__(self, feature_index = None, threshold = None, left = None, right = None, info_gain = None, value = None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        self.value = value
        
class DecisionTreeClassifier():
    def __init__(self, min_sample_split = 2, max_depth = 2):
        self.root = None
        self.min_sample_split = min_sample_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth = 0):
        X, y = dataset.data, dataset.target
        num_samples, num_features = np.shape(X)
         
        if num_samples >= self.min_sample_split and curr_depth <= self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            if best_split["info_gain"] > 0 :
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth + 1)
                right_subtree  = self.build_tree(best_split["dataset_right"], curr_depth + 1)
                return Node(best_split["feature_index"], best_split["threshold"], left_subtree, right_subtree, best_split["info_gain"])
        leaf_value  = self.calculate_leaf_value(y)
        return Node(value = leaf_value)
    
    
    def get_best_split(self, dataset, num_samples, num_features):
        best_split = {}
        max_info_gain = -float("inf")
        for feature_index in range(num_features):
            feature_value = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_value)
            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left) > 0  and len(dataset_right) > 0:
                    y, left_y, right_y = dataset.target[:-1], dataset_left.target[:-1], dataset_right.target[:-1]
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    if curr_info_gain > max_info_gain:
                        max_info_gain = curr_info_gain
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        dataset_left = np.array([row for row in dataset.data if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset.data if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def infomation_gain(self, parent, l_child, r_child, mode = "entropy"):
        weight_l = len(l_child)/len(parent)
        weight_r = len(r_child)/len(parent)
        gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        

IndentationError: expected an indented block (2009736818.py, line 62)